In [ ]:
from flask import Flask, jsonify, send_file, request
import pandas as pd
import mariadb
import re
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from flask_cors import CORS
from konlpy.tag import Okt
import os

app = Flask(__name__)
CORS(app)

# MariaDB 연결 함수
def get_db_connection():
    conn = mariadb.connect(
        user="root",
        password="1234",
        host="localhost",
        port=3306,
        database="ansan"
    )
    return conn

korean_stopwords = set([
    '있다', '하다', '되다', '수', '을', '에', '의', '가', '이', '으로', '도', '를', '는', '다', '그', '그리고', '에서',
    '지음', '있는', '통해', '위해', '대한', '가장', '모든', '한다', '라는', '여러', '같은', '많은', '대해'
])

okt = Okt()

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # 특수 문자 제거
    words = okt.nouns(text)  # 명사 추출
    words = [word for word in words if word not in korean_stopwords and len(word) > 1]  # 한글 불용어 및 1글자 이하 단어 제거
    return words

def create_wordcloud(id):
    try:
        conn = get_db_connection()
        query = f"SELECT book_detail FROM book WHERE id={id}"
        df = pd.read_sql(query, conn)
        conn.close()

        if df.empty:
            return None

        text = df.loc[0, 'book_detail']
        processed_text = preprocess_text(text)

        word_freq = Counter(processed_text)

        font_path = "D:\\python\\NanumGothic.ttf"

        wordcloud = WordCloud(width=800, height=400, background_color='white', font_path=font_path).generate_from_frequencies(word_freq)
        wordcloud_path = f"D:\\python\\wordcloud_{id}.png"
        wordcloud.to_file(wordcloud_path)
        return wordcloud_path
    except Exception:
        return None

@app.route('/api/wordcloud/<int:id>', methods=['GET'])
def get_wordcloud(id):
    wordcloud_path = create_wordcloud(id)
    if wordcloud_path and os.path.exists(wordcloud_path):
        return send_file(wordcloud_path, mimetype='image/png')
    else:
        return '', 204

if __name__ == '__main__':
    app.run(port=5001)
